In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
import glob
from PIL import Image

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [15]:
NUM_CHANNELS = 3
LEARNING_RATE = 1e-3
EPOCHS = 25
BATCH_SIZE = 8
IMAGE_HEIGHT = 1080
IMAGE_WIDTH = 1920
data = []

In [16]:
# filepaths
tray1_rgb = "image_data/masks/T01_rgb"
tray1_mask = "image_data/masks/T01_mask"

In [17]:
'''def dataloader(filepath1, filepath2):
    imageset1 = []
    imageset2 = [] 
    with open(filepath1, "r") as file:
        images = file.readlines()
        for img in images:
            image_filename = img.strip()
            image_path = os.path.join(filepath1, image_filename)
            image = Image.open(image_path)
            imageset1.append(image)
        file.close
    for img in imageset1:
        with open(filepath2, "r") as file:
            images = file.readlines()
            for img in images:
                image_filename = img.strip()
                image_path = os.path.join(filepath2, image_filename)
                image = Image.open(image_path)
                imageset1.append(image)
        file.close

    return imageset1, imageset2'''


'def dataloader(filepath1, filepath2):\n    imageset1 = []\n    imageset2 = [] \n    with open(filepath1, "r") as file:\n        images = file.readlines()\n        for img in images:\n            image_filename = img.strip()\n            image_path = os.path.join(filepath1, image_filename)\n            image = Image.open(image_path)\n            imageset1.append(image)\n        file.close\n    for img in imageset1:\n        with open(filepath2, "r") as file:\n            images = file.readlines()\n            for img in images:\n                image_filename = img.strip()\n                image_path = os.path.join(filepath2, image_filename)\n                image = Image.open(image_path)\n                imageset1.append(image)\n        file.close\n\n    return imageset1, imageset2'

In [18]:
def dataloader(X_filepath, y_filepath):
    X_dir = X_filepath
    y_dir = y_filepath
    X_files = glob.glob(os.path.join(X_dir, '*.jpg'))
    y_files = glob.glob(os.path.join(y_dir, '*.jpg'))
    X = []
    y = []

    for image in X_files:
        X.append(Image.open(image))
        temp_X = image
        for y_image in y_files:
            if (temp_X == y_image):
                y.append(y_image)
    

    return X,y

In [19]:
X, y = dataloader(tray1_rgb, tray1_mask)
print(X)

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC244C18B0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC2C5FB7C0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC2C5FB190>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC2C5FB7F0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC2C5F0B20>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC2C5F09D0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC2C5F0520>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC2C5F0910>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC2C5F0B80>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC2C5F05B0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x7FEC2C5F0FA0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080

In [20]:
class conv_block(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, output_channels, kernel_size = 3, stride =1, padding=1)
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.conv2 = nn.Conv2d(output_channels, output_channels, kernel_size = 3, stride = 1, padding=1)
        self.bn2 = nn.BatchNorm2d(output_channels)
        self.relu = nn.ReLU()

        def forward(self, input):
            x = self.conv1(input)
            x = self.bn1(x)
            x = self.conv2(x)
            x = self.bn2(x)
            x = self.relu(x)
            return x

In [21]:
class encoder_block(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv = conv_block(input_channels, output_channels)
        self.pool = nn.MaxPool2d((2,2))

    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)
        return x, p

In [22]:
class decoder_block(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(input_channels, output_channels, kernel_size=3, stride=1, padding=1)
        self.conv = conv_block(output_channels*2, output_channels)
        
    def forward(self, inputs, skip):
        x = self.up(inputs)
        x = torch.cat([x, skip], axis=1)
        x = self.conv(x)
        return x

In [23]:
class UNET(nn.Module):
    def __init__(self):
        super().__init__()

        self.e1 = encoder_block(3, 64)
        self.e2 = encoder_block(64, 128)
        self.e3 = encoder_block(128, 256)
        self.e4 = encoder_block(256, 512)

        self.b = conv_block(512,1024)

        self.d1 = decoder_block(1024, 512)
        self.d2 = decoder_block(512, 256)
        self.d3 = decoder_block(256, 128)
        self.d4 = decoder_block(128, 64)

        self.outputs = nn.Conv2d(64, 1, kernel_size=1, padding=0)

    def forward(self, inputs):

        s1, p1 = self.e1(inputs)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)

        b = self.b(p4)

        d1 = self.d1(b, s4)
        d2 = self.d2(d1, s3)
        d3 = self.d3(d2, s2)
        d4 = self.d4(d3, s1)

        outputs = self.outputs(d4)

        return outputs

In [ ]:
def train_loop():
    

In [24]:
model = UNET()
print(model)

UNET(
  (e1): encoder_block(
    (conv): conv_block(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (e2): encoder_block(
    (conv): conv_block(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (pool): MaxPool2d(kernel_size=(2, 2), str